In [4]:
l = [['ola'],['adeus']]

if ['ola'] in l:
    print('oi')

oi


In [12]:
import requests
import json

from sentence_transformers import SentenceTransformer

from sklearn.metrics.pairwise import cosine_similarity

ModuleNotFoundError: No module named 'torch._C'

In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

characters = ['Monica Geller', 'Joey Tribbiani', 'Chandler Bing', 'Phoebe Buffay', 'Ross Geller', 'Rachel Green']


sentences = []

for character in characters:
    f = open(character + ".txt", "r")
    lines = f.readlines()
    sentences.append(lines)


embedding_monica = model.encode(sentences[0])
embedding_joey = model.encode(sentences[1])
embedding_chandler = model.encode(sentences[2])
embedding_phoebe = model.encode(sentences[3])
embedding_ross = model.encode(sentences[4])
embedding_rachel = model.encode(sentences[5])


NameError: name 'SentenceTransformer' is not defined

In [ ]:
seasons = []

for i in range(1,11):

    # load season i
    if i<10:
        json_file = 'https://raw.githubusercontent.com/emorynlp/character-mining/master/json/friends_season_0'+str(i)+'.json'
    else:
        json_file = 'https://raw.githubusercontent.com/emorynlp/character-mining/master/json/friends_season_10.json'
    
    r = requests.get(json_file)
    seasons.append(json.loads(r.text))

In [ ]:
def compute_similarity(vector):
    cos_monica = cosine_similarity(vector, embedding_monica)
    cos_joey = cosine_similarity(vector, embedding_joey)
    cos_chandler = cosine_similarity(vector, embedding_chandler)
    cos_phoebe = cosine_similarity(vector, embedding_phoebe)
    cos_ross = cosine_similarity(vector, embedding_ross)
    cos_rachel = cosine_similarity(vector, embedding_rachel)

    #Smaller angles between vectors produce larger cosine values, indicating greater cosine similarity


In [ ]:
test_set = {}

for season in seasons:
    for episode in season['episodes']:
        for scene in episode['scenes']:
            for utterance in scene['utterances']:
                if utterance['speakers'] != [] and utterance not in sentences:
                    test_set[utterance['speakers']] = utterance


for line in test_set.items():
    line_embed = model.encode(line)

    predicted = compute_similarity(line_embed)

    print("predicted :"predicted)
    print("real:" line['speakers'])
